In [266]:
pip install yfinance numpy pandas matplotlib backtesting

Note: you may need to restart the kernel to use updated packages.


Iteration 1: Backtest the SMA moving average strategy (Golden Cross)
Guide from: 
Part Time Larry: Backtesting.py from Youtube
Backtesting Tutorial from https://kernc.github.io/backtesting.py/doc/examples/Strategies%20Library.html

In [267]:
import yfinance as yf
import numpy as np
import pandas as pd
import math
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [268]:
#stock data
pltr = yf.Ticker("PLTR") #anything else is applicable
data = pltr.history(period="5y")
print(data.to_string())

                                 Open        High         Low       Close     Volume  Dividends  Stock Splits
Date                                                                                                         
2020-11-20 00:00:00-05:00   19.030001   19.660000   18.000000   18.150000   72104000        0.0           0.0
2020-11-23 00:00:00-05:00   18.629999   21.450001   18.570000   21.040001   84716300        0.0           0.0
2020-11-24 00:00:00-05:00   22.350000   24.270000   20.629999   23.820000  115468800        0.0           0.0
2020-11-25 00:00:00-05:00   23.959999   29.750000   23.690001   29.049999  202563700        0.0           0.0
2020-11-27 00:00:00-05:00   31.190001   33.500000   26.120001   27.660000  207834000        0.0           0.0
2020-11-30 00:00:00-05:00   28.240000   30.150000   24.110001   27.110001  137243000        0.0           0.0
2020-12-01 00:00:00-05:00   28.090000   28.139999   24.450001   25.670000   84050000        0.0           0.0
2020-12-02

In [274]:
from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting.test import SMA

def SMA(values, n): #moving average
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    n1 = 50
    n2 = 200
    
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        if crossover(self.sma1, self.sma2): #crossover function
            self.position.close()
            self.buy(size=.95)
            
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [275]:
class SmaCrossSl(TrailingStrategy):
    n1 = 50; n2 = 200
    trailing_sl = 0.01
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.set_trailing_sl(self.trailing_sl)
        
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close(); self.buy(size=0.95)
        elif crossover(self.sma2, self.sma1):
            self.position.close(); self.sell()

In [276]:
from backtesting import Backtest

bt = Backtest(data, SmaCross, cash=100_000, commission=.002)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/1055 [00:00<?, ?bar/s]

C:\Users\USER\AppData\Local\Temp\ipykernel_27680\1283243814.py:4: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = bt.run()


Start                     2020-11-20 00:00...
End                       2025-11-19 00:00...
Duration                   1825 days 00:00:00
Exposure Time [%]                    29.96016
Equity Final [$]                 2461337.9407
Equity Peak [$]                 3081139.77917
Commissions [$]                      560.2651
Return [%]                         2361.33794
Buy & Hold Return [%]               546.67709
Return (Ann.) [%]                    90.25792
Volatility (Ann.) [%]               103.53358
CAGR [%]                             55.63095
Sharpe Ratio                          0.87177
Sortino Ratio                         3.06413
Calmar Ratio                          2.22989
Alpha [%]                          2156.78263
Beta                                  0.37418
Max. Drawdown [%]                   -40.47634
Avg. Drawdown [%]                    -7.10205
Max. Drawdown Duration      223 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                          

In [277]:
bt2 = Backtest(data, SmaCrossSl, cash=100_000, commission=.002)
stats2 = bt2.run()
stats2

Backtest.run:   0%|          | 0/1055 [00:00<?, ?bar/s]

C:\Users\USER\AppData\Local\Temp\ipykernel_27680\1790017312.py:2: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats2 = bt2.run()


Start                     2020-11-20 00:00...
End                       2025-11-19 00:00...
Duration                   1825 days 00:00:00
Exposure Time [%]                    29.96016
Equity Final [$]                 2461337.9407
Equity Peak [$]                 3081139.77917
Commissions [$]                      560.2651
Return [%]                         2361.33794
Buy & Hold Return [%]               546.67709
Return (Ann.) [%]                    90.25792
Volatility (Ann.) [%]               103.53358
CAGR [%]                             55.63095
Sharpe Ratio                          0.87177
Sortino Ratio                         3.06413
Calmar Ratio                          2.22989
Alpha [%]                          2156.78263
Beta                                  0.37418
Max. Drawdown [%]                   -40.47634
Avg. Drawdown [%]                    -7.10205
Max. Drawdown Duration      223 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                          

In [278]:
bt.plot()

c:\anaconda\Lib\site-packages\bokeh\util\serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


GridPlot(id='p13316', ...)